In [1]:
# [v] Soal
# Gambar -> GPT4 -> Text (soal)
# Text (soal) -> speech

# [v] Penyandang
# Speech -> text (jawaban)

# Auto graded AI
# Text (soal) v.stext (jawaban)


# Kamu adalah seorang guru di sekolah luar biasa (SLB). Saat ini kamu sedang mengadakan ujian. Peserta didik memiliki keterbatasan penglihatan (tuna netra). Oleh karena itu, kamu harus menjelaskan soal ujian berikut ini dalam bentuk narasi.

In [22]:
import os
import base64
import requests

from dotenv import load_dotenv

load_dotenv('../.env')

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')

# Image to Text

In [15]:
def explain(encoded_image):
  headers = {
      "Content-Type": "application/json",
      "api-key": AZURE_OPENAI_API_KEY,
  }

  # Payload for the request
  payload = {
    "messages": [
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "Kamu adalah seorang guru di sekolah luar biasa (SLB). Saat ini kamu sedang mengadakan ujian. Peserta didik memiliki keterbatasan penglihatan (tuna netra). Oleh karena itu, kamu harus menjelaskan soal ujian berikut ini dalam bentuk narasi.\n\nIngat, hanya sampaikan soal ujian nya saja, tidak perlu di jawab."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{encoded_image}"
            }
          }
        ]
      },
    ],
    "temperature": 0.5,
    "top_p": 0.95,
    "max_tokens": 800
  }

  # Send request
  try:
      GPT4V_ENDPOINT = f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"
      response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
      response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
  
  except requests.RequestException as e:
      raise SystemExit(f"Failed to make the request. Error: {e}")

  res = response.json()
  return res['choices'][0]['message']['content']

In [10]:
# Configuration
IMAGE_PATH = "../images/soal-ujian/matematika-01.jpeg"
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

In [20]:
explanation = explain(encoded_image)
explanation

'Soal nomor 6. Diketahui persegi panjang pada gambar di bawah ini.\n\nGambar menunjukkan sebuah persegi panjang dengan sisi-sisi yang ditandai sebagai berikut: sisi AB dan sisi CD masing-masing memiliki panjang 24 cm, sedangkan sisi AC dan sisi BD masing-masing memiliki panjang 7 cm.\n\nPertanyaan: Hitunglah keliling bangun tersebut.\n\nPilihan jawaban:\na. 72 cm\nb. 62 cm\nc. 52 cm\nd. 82 cm\ne. 42 cm'

In [25]:
print(explanation)

Soal nomor 6. Diketahui persegi panjang pada gambar di bawah ini.

Gambar menunjukkan sebuah persegi panjang dengan sisi-sisi yang ditandai sebagai berikut: sisi AB dan sisi CD masing-masing memiliki panjang 24 cm, sedangkan sisi AC dan sisi BD masing-masing memiliki panjang 7 cm.

Pertanyaan: Hitunglah keliling bangun tersebut.

Pilihan jawaban:
a. 72 cm
b. 62 cm
c. 52 cm
d. 82 cm
e. 42 cm


# Text to Speech

In [19]:
import os
import time

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import azure.cognitiveservices.speech as speechsdk

In [23]:
language_key = os.environ.get('LANGUAGE_KEY')
language_endpoint = os.environ.get('LANGUAGE_ENDPOINT')

speech_key = "232d072838de4797908c738708459c41"
service_region = "southeastasia"
timestr = time.strftime("%Y%m%d-%H%M%S")
audio_path = f"outputs-speech-{timestr}.wav"
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

if not language_key or not language_endpoint:
    raise ValueError("Please set the LANGUAGE_KEY and LANGUAGE_ENDPOINT environment variables.")

# Fungsi untuk membaca isi file teks dan mengembalikannya sebagai string
def txt_to_string(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Path ke file teks
file_path = "./test2.png.txt"

# Membaca isi file teks
# text = txt_to_string(file_path)
text = explanation

# Authenticate the client using your key and endpoint
def authenticate_client():
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=language_endpoint,
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Example method for detecting the language of text
def language_detection(client, text):
    try:
        # Memanggil metode detect_language dengan teks sebagai list of documents
        response = client.detect_language(documents=[text], country_hint='us')[0]
        return response.primary_language.iso6391_name

    except Exception as err:
        print("Encountered exception. {}".format(err))
    return response.primary_language.iso6391_name
detected_language = language_detection(client,text)

# Choose voice based on detected language
voice_map = {
    'id': 'id-ID-GadisNeural',       # Bahasa Indonesia
    'en': 'en-US-GuyNeural',         # Bahasa Inggris (AS)
    'es': 'es-MX-JorgeNeural',       # Bahasa Spanyol (Meksiko)
    'fr': 'fr-FR-DeniseNeural',      # Bahasa Prancis (Prancis)
    'de': 'de-DE-ConradNeural',      # Bahasa Jerman (Jerman)
    'ar': 'ar-SA-HamedNeural',       # Bahasa Arab (Arab Saudi)
    'zh_chs': 'zh-CN-XiaoxiaoNeural',    # Bahasa Mandarin (China)
    'ja': 'ja-JP-NanamiNeural',      # Bahasa Jepang
    'pt': 'pt-BR-FranciscaNeural',   # Bahasa Portugis (Brasil)
    'ru': 'ru-RU-SvetlanaNeural',    # Bahasa Rusia
    'nl': 'nl-NL-ColetteNeural',     # Bahasa Belanda
    'ko': 'ko-KR-SunHiNeural',       # Bahasa Korea
    'tr': 'tr-TR-EmelNeural',        # Bahasa Turki
    'sv': 'sv-SE-HilleviNeural',     # Bahasa Swedia
    'pl': 'pl-PL-ZofiaNeural',       # Bahasa Polandia
    'cs': 'cs-CZ-VlastaNeural',      # Bahasa Ceska (Ceko)
    'hu': 'hu-HU-NoemiNeural',       # Bahasa Hungaria
    'ro': 'ro-RO-EmilNeural',        # Bahasa Rumania
    'sk': 'sk-SK-LukasNeural',       # Bahasa Slovak
    'bg': 'bg-BG-BorislavNeural',    # Bahasa Bulgaria
    'hr': 'hr-HR-GabrijelaNeural',   # Bahasa Kroasia
    'lv': 'lv-LV-NilsNeural',        # Bahasa Latvia
    'it':'it-IT-ElsaNeural',         # Bahasa Italia
    'vi':'vi-VN-HoaiMyNeural'        # Bahsa Vietnam
            }
    # Add more languages and voices as needed

# Fallback to a default voice if language is not in the map
voice = voice_map.get(detected_language)
# Note: the voice setting will not overwrite the voice element in input SSML.
speech_config.speech_synthesis_voice_name = voice
audio_config = speechsdk.audio.AudioOutputConfig(filename=audio_path)
# use the default speaker as audio output.
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
result = speech_synthesizer.speak_text_async(text).get()

In [ ]:
speech_key = "232d072838de4797908c738708459c41"
service_region = "southeastasia"
timestr = time.strftime("%Y%m%d-%H%M%S")
audio_path = f"./../Audio/outputs-speech-{timestr}.wav"
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)


In [24]:
result

Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: received close frame, sending a close response frame.
Info: on_underlying_io_close_sent: uws_client=0x115f85c10, io_send_result:0
Info: on_underlying_io_close_sent: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.


# Speech to Text

In [26]:
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

In [31]:
import azure.cognitiveservices.speech as speechsdk

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
source_language_config = speechsdk.languageconfig.SourceLanguageConfig("id-ID")  
audio_config = speechsdk.audio.AudioConfig(filename='../audios/jawaban.wav')


speech_recognizer = speechsdk.SpeechRecognizer(  
    speech_config=speech_config, 
    source_language_config=source_language_config, 
    audio_config=audio_config
)

result = speech_recognizer.recognize_once()

print(result.text)

Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.
Jawabannya adalah 62cm.


In [33]:
answer = result.text
answer

'Jawabannya adalah 62cm.'

# Auto Graded

In [32]:
explanation

'Soal nomor 6. Diketahui persegi panjang pada gambar di bawah ini.\n\nGambar menunjukkan sebuah persegi panjang dengan sisi-sisi yang ditandai sebagai berikut: sisi AB dan sisi CD masing-masing memiliki panjang 24 cm, sedangkan sisi AC dan sisi BD masing-masing memiliki panjang 7 cm.\n\nPertanyaan: Hitunglah keliling bangun tersebut.\n\nPilihan jawaban:\na. 72 cm\nb. 62 cm\nc. 52 cm\nd. 82 cm\ne. 42 cm'

In [34]:
answer

'Jawabannya adalah 62cm.'

In [38]:
from openai import AzureOpenAI

# endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
deployment = "gpt-35-turbo" 

tools = [
    {
        "type": "function",
        "function": {
            "name": "berikan_penilaian",
            "description": "Berika penilaian",
            "parameters": {
                "type": "object",
                "properties": {
                    "benar_salah": {
                        "type": "boolean",
                        "description": "Apakah jawaban pengguna benar atau salah",
                    },
                    "penjelasan": {
                        "type": "string",
                        "description": "Jelaskan jawaban yang benar",
                    },
                },
                "required": ["benar_salah", "penjelasan"],
            },
        },
    }
]

client = AzureOpenAI(
    api_version="2024-02-01",
)

SYSTEM_PROMPT = '''
Kamu adalah seorang guru yang saat ini sedang mengadakan ujian. Soal berupa pilihan ganda. 
Berikut adalah solanya:
'''
completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "system",
            "content": """Kamu adalah seorang guru yang saat ini sedang mengadakan ujian. Soal berupa pilihan ganda. 
Berikut adalah solanya:""",
        },
        {
            "role": "system",
            "content": explanation,
        },
        {
            "role": "user",
            "content": answer
        },
        {
            "role": "assistant",
            "content": """
Berdasarkan jawaban pengguna di atas ini, berikan penilaian dalam format berikut:
Jawaban: BENAR atau SALAH
Penjelasan: Jelaskan jawaban yang benar"""
        },
    ],
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "berikan_penilaian"}}
)
      
# print(completion.to_json())

In [45]:
completion.choices[0].message.tool_calls[0].function.name

'berikan_penilaian'

In [47]:
import json
args = json.loads(completion.choices[0].message.tool_calls[0].function.arguments)
args

{'benar_salah': True,
 'penjelasan': 'Keliling persegi panjang dapat dihitung dengan menjumlahkan panjang semua sisinya. Pada gambar tersebut, sisi AB dan sisi CD memiliki panjang 24 cm, sedangkan sisi AC dan sisi BD memiliki panjang 7 cm. Jadi, kelilingnya adalah 2 × (24 cm + 7 cm) = 62 cm.'}

In [ ]:
is_correct = args['benar_salah']
explanation = args['penjelasan']
